# Docker란?

- https://docs.docker.com/reference/cli/docker/

## 1. Docker Command

* docker [container] [start/stop/create/run/rm/exec/ps/cp/commit] [--name/-e/-p/-v/-d/-i/-t/-a]
* docker [image] [pull/rm/ls/build] [-i]
* docker [volume] [create/inspect/ls/prune/rm] [--name/-a]
* docker [network] [connect/disconnect/create/cl/ls/prune/rm] []

## 2. option

* --name : 컨테이너/네트워크등 이름
* -p(--publish) : 호스트포트번호:컨테이너포트 
* -v(--volume) : 호스트저장소(디렉토리):컨테이너저장소(디렉토리)
* -net : 네트워크이름
* -e(--env) : 환경변수값(예: MYSQL_PASSWORD...)
* -d(--detach) : 백그라운드로 실행
* -i(--interactive) : 컨테이너에 터미널(커맨드창)로 연결
* -t(-tty) : 컨테이너를 터미널환경에서 사용여부
* --help : 도움말


* docker run -dit --name mysql01 -p 3316:3306 -e MYSQL_ROOT_PASSWORD=12345 -e MYSQL_DATABASE=board -e MYSQL_USER=user01 -e MYSQL_PASSWORD=abcde mysql

* docker network create mynetwork
* docker network ls
* docker mynetwork inspect

* docker run -dit --name mysql02 -p 3317:3306 --net=mynetwork -v d:/temp:/var/lib/mysql -e MYSQL_ROOT_PASSWORD=12345 -e MYSQL_DATABASE=board -e MYSQL_USER=user01 -e MYSQL_PASSWORD=abcde mysql

# 1. 도커/호스트 양방향 파일복사  

## 1.1 도커이미지에서 호스트로 또는 그 반대로 복사

- 볼륨에서도 양방향 복사
- httpd 웹서버를 만들어서 index.html을 변경(복사) 

- docker run --name myweb01 -d -p 8090:80 httd
- docker cp d:\lec\07.docker\index.html myweb01:/usr/local/apache2/htdocs/
- docker cp myweb01:/usr/local/apache2/htdocs/index.html d:\lec\07.docker\

## 1.2 볼륨(호스트)과 양방향복사

- docker volume cerate myvol01
- docker run --name myweb02 - d - p 8091:80 -v myvol01:/usr/local/apache2/htdocs/ httpd


# 2. 도커이미지생성 및 업로드
- myweb01 컨테이너를 이미지로 생성 및 업로드(hub.docker.com)

- docker commit myweb01 your도커아이디/myimage01:v0.2
- docker push your도커아이디/myimage01:v0.2
- docker pull your도커아이디/myimage01:v0.2
- docker run --name myweb03 -d -p 8092:80 your도커아이디/myimage01:v0.2

# 4. 도커이미지내에서 실행하기

- docker run --name myubuntu -dit ubuntu
- docker exec -it myubuntu /bin/bash
  - JDK 프로그램설치 
  - root사용자가 아닐 경우 sudo 실행파일...
  - root사용자일 경우에는 sudo없이 바로 실행파일...

> 우분투에서 패키지를 설치하는 명령
>>$ [sudo] apt-get update
>>$ [sudo] apt-get install -y openjdk-17-jdk
>>$ [sudo] apt-get install vim
>>  - vi Main.java
>>  - cat Main.java
>>  - javac Main.java
>>  - chmod +x Main.class
>>  - java -jar Main

## 4.1 Board SpringBoot Webapp 생성

- gradlew clean build

## 4.2 BoardWebapp를 ubuntu 루트폴더에 복사

- docker cp D:\lec\07.docker\sp01_docker\build\libs\app.jar myubuntu:/

## 4.3 BoardWebapp 이미지 빌드
- docker commit myubuntu mywebserver

## 4.4 BoardWebapp 컨테이너 생성
- docker run --name myboard01 -dit -p 9091:8090 mywebserver 
- docker run --name myboard02 -dit -p 9092:8090 mywebserver 
- docker run --name myboard03 -dit -p 9093:8090 mywebserver 
- docker run --name myboard04 -dit -p 9094:8090 -p 9095:8090 mywebserver 


# 5. Dockerfile 

- Dockerfile은 docker이미지를 생성하기 위한 스크립트이다.
- Dockerfile에 나열된 명령문을 순차적으로 수행해서 최종 산출물로 도커이미지를 생성해 준다.
- 도커파일을 작성할 때 실제 파일의 이름은 `Dockerfile`로 해야 한다.

## 5.1 Dockerfile 작성 및 명령

>* FROM: 베이스이미지
>* RUN: 도커이미지에서 명령을 실행
>* WORKDIR: 작업디렉토리를 지정, 없다면 디렉토리를 생성
>* EXPOSE: 빌드로 생성된 이미지의 포트를 오픈
>* USER: 이미지를 어떤 사용자로 실행할 것인지를 정의, 기본값은 root
>* COPY/ADD: build명령중간에 호스트의 파일 or 폴더를 복사(추가)
>> - ADD명령은 COPY보다 좀 더 강력한 명령문
>> - COPY는 파일 or 폴더를 복사할 수 있지만 
>> - ADD는 일반파일뿐만 아니라 압축파일 or 네트워크파일도 복사가능
>* ENV : 이미지에서 사용할 환경변수값 정의
>* CMD/ENTRYPOINT : 컨테이너를 생성 및 실행할 때 실행할 명령어
>> - 보통 컨테이너 내부에서 항상 실행해야 하는 서버를 구동할 때 사용한다.
>> - CMD
>>>  - 컨테이너를 실행할 때만 실행된다.(docker run)
>>>  - 컨테이너생성시, 추가적인 명령어에 따라 설정한 명령을 수정할 수 있다.
>> - ENTRYPOINT
>>>  - 컨테이너를 시작할 때마다 실행된다(docker start)
>>>  - 컨테이너시작시, 추가적인 명령어 존재여부와 상관없이 무조건 실행한다.
>> - 명령어형식
>>>  - CMD ["커맨드", "파라미터1", ... "파라미터n"]
>>>  - CMD 커맨드, 파라미터1, ... 파라미터n
>>>  - ENTRYPOINT ["커맨드", "파라미터1", ... "파라미터n"]
>>>  - ENTRYPOINT 커맨드, 파라미터1, ... 파라미터n 

## 5.2 Dockerfile로 이미지생성

```bash
$ docker build -t 이미지이름:버전 도커파일경로

$ docker build -t boardweb:v1.0 .(현재경로)
$ docker build -t boardweb:v1.0 ./sp01_docker
```

## 5.3 Dockerfile 실습

```bash
FROM ubuntu
COPY build/libs/*.jar app.jar
RUN apt-get update && apt-get install -y \
    openjdk-17-jdk
EXPOSE 8090
CMD ["java", "-jar", "./app.jar"]
```

# 6. docker-compose

- 도커컴포즈는 단일서버에서 여러개의 컨테이너를 하나의 서비스로 정의해서 일괄로 관리할 수 있게 해준다.
- 복수의 컨테이너가 하나의 어플리케이션으로 도작할 때 도커컴포즈를 사용하지 않는다면 이를 테스트하기 위해서는
- 각 컨테이너를 하나씩 생성해야 한다. 
- 이와같은 방법은 일일이 여러개의 컨테이너를 실행하기에는 너무 번거롭기 때문에 복수의 컨테이너를 하나의 서비스로
- 정리해서 일괄로 관리할 수 있다.
- 도커컴포즈는 복수의 컨테이너를 순차적으로 생성하는 방식으로 동작한다.
- 도커컴포즈의 설정파일은 도커의 run명령어 옵션을 그대로 사용하구 있고 각 컨테니너의 의존성을 정의할 수 있다.
- 즉, board웹에서 데이터베이스(mysql) or 네트워크를 사용할 것인지 의존성을 정의할 수 있다.


```bash
# 도커컴포즈버전확인
$ docker-compose -v
```

## 6.1 docker compose 속성

>* version
>> 1. 도커컴포즈의 파일포맷 버전을 지정
>> 1. 기본적으로 버전 3을 사용하는 것이 일반적인다.
>* services
>> 1. 서비스의 이름
>* image
>> 1. docker컨테이너의 이미지명을 정의
>> 1. docker hub에 있는 이미지를 사용하여 docker container를 작성할 경우 이미지를 설정할 수 있다.
>* restart
>> 1. 컨테이너가 다운되었을 경우 항상 재시작하라는 명령을 설정
>* volumes
>> 1. 하나의 docker run명령의 -v 옵션과 동일한 역할
>> 1. 여러개의 볼륨을 정의할 수 있으며 리스트처럼 작성하면 된다.
>* environment
>> 1. docker명령의 -e or Dockerfile의 ENV와 동일한 역할
>> 1. env_file옵션으로 환경파일이 설정되어 있는 외부파일을 읽을 수도 있다.(비밀번호 같은 보안을 위한 방법)
>* ports
>> 1. docker run명령의 -p와 동일한 역할
>> 1. 8090:8090 or 8081-8099등의 형식으로 정의가능
>* build
>> 1. Dockerfile을 기반으로 이미지를 생성할 경우 사용
>* depends_on
>> 1. 컨터이너가 어떤 컨테이너를 의존하는지를 설정
>> 1. 예를 들어 boardweb안에 `depends_on: -dba: mysql`으로 설정되어 있다면 
>> 1. 우선적으로 depends_on에 정의된 컨테이너가 먼저 생성이 되고 난후에
>> 1.  boardapp의 컨테이너가 실행되어 boardweb이 mysql컨테이너에 접속되도록 실행순서를 관리하는 것이다.

## 6.2 docker-compose 명령

```bash
# 컨테이너실행
$ docker-compose up -d                # 도커를 백그라운드로 실행
$ docker-compose up --force-recreate  # 도커컨데이를 새로 생성
$ docker-compose up --build           # 도커이미지를 빌드후 docker-compose up명령실행

# 컨테이너중지
$ docker-compose down # 도커컨테이너를 중지 and 삭제
$ docker-compose stop # 도커컨테이너를 중지
```

## 6.3 도커컴포즈파일(.yml)
-- 작성문법은 yaml파일과 동일하지만 탭키는 사용하지 않고 스페이스2칸으로 들여쓰기를 한다. 
-- docker-compose.yml
```yml
version: "3"
services:
  mysql000ex11:
    image: mysql:8
    ports:
      - 3307:3306
    networks:
      - wordpress000net1
    volumes:
      - mysql000vol11:/var/lib/mysql
    restart: always
    environment:
      MYSQL_ROOT_PASSWORD: myrootpass
      MYSQL_DATABASE: wordpress000db
      MYSQL_USER: wordpress000kun
      MYSQL_PASSWORD: wkunpass
  wordpress000ex12:
    depends_on:
      - mysql000ex11
    image: wordpress
    networks:
      - wordpress000net1
    volumes:
      - wordpress000vol12:/var/www/html
    ports:
      - 8085:80
    restart: always
    environment:
      WORDPRESS_DB_HOST: mysql000ex11
      WORDPRESS_DB_NAME: wordpress000db
      WORDPRESS_DB_USER: wordpress000kun
      WORDPRESS_DB_PASSWORD: wkunpass
networks:
  wordpress000net1: {}
volumes:
  mysql000vol11:
  wordpress000vol12:
```

#### docker-compose 실행

```bash
$ docker-compose -f docker-compose.yml up -d
```